*preset*

In [1]:
"""
[file]          preset.py
[description]   default settings of WiFi-based models
"""
#
##
model123 = "THAT_location"
preset = {
    #
    ## define model
    "model": "LSTM",                                    # "ST-RF", "MLP", "LSTM", "CNN-1D", "CNN-2D", "CLSTM", "ABLSTM", "THAT"
    #
    ## define task
    "task": "location",                                 # "identity", "activity", "location"
    #
    ## number of repeated experiments
    "repeat": 1,
    #
    ## path of data
    "path": {
        "data_x": "/kaggle/input/wimans/wifi_csi/amp",               # directory of CSI amplitude files
        "data_y": "/kaggle/input/wimans/annotation.csv",             # path of annotation file
        "save": f"result_{model123}.json"                           # path to save results
    },
    #
    ## data selection for experiments
    "data": {
        "num_users": ["0", "1", "2", "3", "4", "5"],    # select number(s) of users, (e.g., ["0", "1"], ["2", "3", "4", "5"])
        "wifi_band": ["2.4"],                           # select WiFi band(s) (e.g., ["2.4"], ["5"], ["2.4", "5"])
        "environment": ["empty_room"],                   # select environment(s) (e.g., ["classroom"], ["meeting_room"], ["empty_room"])
        "length": 3000,                                 # default length of CSI
    },
    #
    ## hyperparameters of models
    "nn": {
        "lr": 1e-3,                                     # learning rate
        "epoch": 300,                                   # number of epochs
        "batch_size": 128,                              # batch size
        "threshold": 0.5,                               # threshold to binarize sigmoid outputs
    },
    #
    ## encoding of activities and locations
    "encoding": {
        "activity": {                                   # encoding of different activities
            "nan":      [0, 0, 0, 0, 0, 0, 0, 0, 0],
            "nothing":  [1, 0, 0, 0, 0, 0, 0, 0, 0],
            "walk":     [0, 1, 0, 0, 0, 0, 0, 0, 0],
            "rotation": [0, 0, 1, 0, 0, 0, 0, 0, 0],
            "jump":     [0, 0, 0, 1, 0, 0, 0, 0, 0],
            "wave":     [0, 0, 0, 0, 1, 0, 0, 0, 0],
            "lie_down": [0, 0, 0, 0, 0, 1, 0, 0, 0],
            "pick_up":  [0, 0, 0, 0, 0, 0, 1, 0, 0],
            "sit_down": [0, 0, 0, 0, 0, 0, 0, 1, 0],
            "stand_up": [0, 0, 0, 0, 0, 0, 0, 0, 1],
        },
        "location": {                                   # encoding of different locations
            "nan":  [0, 0, 0, 0, 0],
            "a":    [1, 0, 0, 0, 0],
            "b":    [0, 1, 0, 0, 0],
            "c":    [0, 0, 1, 0, 0],
            "d":    [0, 0, 0, 1, 0],
            "e":    [0, 0, 0, 0, 1],
        },
    },
}


pre process

In [2]:
"""
[file]          preprocess.py
[description]   preprocess WiFi CSI data
"""
#
##
import os
import argparse
import numpy as np
import scipy.io as scio

#
##
def mat_to_amp(data_mat):
    """
    [description]
    : calculate amplitude of raw WiFi CSI data
    [parameter]
    : data_mat: dict, raw WiFi CSI data from *.mat files
    [return]
    : data_csi_amp: numpy array, CSI amplitude
    """
    #
    ## 
    var_length = data_mat["trace"].shape[0]
    #
    data_csi_amp = [abs(data_mat["trace"][var_t][0][0][0][-1]) for var_t in range(var_length)]
    #
    data_csi_amp = np.array(data_csi_amp, dtype = np.float32)
    #
    return data_csi_amp

#
##
def extract_csi_amp(var_dir_mat, 
                    var_dir_amp):
    """
    [description]
    : read raw WiFi CSI files (*.mat), calcuate CSI amplitude, and save amplitude (*.npy)
    [parameter]
    : var_dir_mat: string, directory to read raw WiFi CSI files (*.mat)
    : var_dir_amp: string, directory to save WiFi CSI amplitude (*.npy)
    """
    #
    ##
    var_path_mat = os.listdir(var_dir_mat)
    #
    for var_c, var_path in enumerate(var_path_mat):
        #
        data_mat = scio.loadmat(os.path.join(var_dir_mat, var_path))
        #
        data_csi_amp = mat_to_amp(data_mat)
        #
        
        #
        var_path_save = os.path.join(var_dir_amp, var_path.replace(".mat", ".npy"))
        #
        with open(var_path_save, "wb") as var_file:
            np.save(var_file, data_csi_amp)

#
##
def parse_args():
    """
    [description]
    : parse arguments from input
    """
    #
    ##
    var_args = argparse.ArgumentParser()
    #
    var_args.add_argument("--dir_mat", default = "/kaggle/input/wimans/wifi_csi/mat", type = str)
    var_args.add_argument("--dir_amp", default = "/kaggle/input/wimans/wifi_csi/amp", type = str)
    #
    return var_args.parse_args()



**load data**

In [3]:
"""
[file]          load_data.py
[description]   load annotation file and CSI amplitude, and encode labels
"""
#
##
import os
import numpy as np
import pandas as pd
#


#
##
def load_data_y(var_path_data_y,
                var_environment = None, 
                var_wifi_band = None, 
                var_num_users = None):
    """
    [description]
    : load annotation file (*.csv) as a pandas dataframe
    : according to selected environment(s), WiFi band(s), and number(s) of users
    [parameter]
    : var_path_data_y: string, path of annotation file
    : var_environment: list, selected environment(s), e.g., ["classroom"]
    : var_wifi_band: list, selected WiFi band(s), e.g., ["2.4"]
    : var_num_users: list, selected number(s) of users, e.g., ["0", "1", "2"]
    [return]
    : data_pd_y: pandas dataframe, labels of selected data
    """
    #
    ##
    data_pd_y = pd.read_csv(var_path_data_y, dtype = str)
    #
    if var_environment is not None:
        data_pd_y = data_pd_y[data_pd_y["environment"].isin(var_environment)]
    #
    if var_wifi_band is not None:
        data_pd_y = data_pd_y[data_pd_y["wifi_band"].isin(var_wifi_band)]
    #
    if var_num_users is not None:
        data_pd_y = data_pd_y[data_pd_y["number_of_users"].isin(var_num_users)]
    #
    return data_pd_y

#
##
def load_data_x(var_path_data_x, 
                var_label_list):
    """
    [description]
    : load CSI amplitude (*.npy)
    : according to a label list of selected data
    [parameter]
    : var_path_data_x: string, directory of CSI amplitude files
    : var_label_list: list, selected labels
    [return]
    : data_x: numpy array, CSI amplitude
    """
    #
    ##
    var_path_list = [os.path.join(var_path_data_x, var_label + ".npy") for var_label in var_label_list]
    #
    data_x = []
    #
    for var_path in var_path_list:
        #
        data_csi = np.load(var_path)
        #
        var_pad_length = preset["data"]["length"] - data_csi.shape[0]
        #
        data_csi_pad = np.pad(data_csi, ((var_pad_length, 0), (0, 0), (0, 0), (0, 0)))
        #
        data_x.append(data_csi_pad)
    #
    data_x = np.array(data_x)
    #
    return data_x

#
##
def encode_data_y(data_pd_y, 
                  var_task):
    """
    [description]
    : encode labels according to specific task
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    : var_task: string, indicate task
    [return]
    : data_y: numpy array, label encoding of task
    """
    #
    ##
    if var_task == "identity":
        #
        data_y = encode_identity(data_pd_y)
    #
    elif var_task == "activity":
        #
        data_y = encode_activity(data_pd_y, preset["encoding"]["activity"])
    #
    elif var_task == "location":
        #
        data_y = encode_location(data_pd_y, preset["encoding"]["location"])
    #
    return data_y

#
##
def encode_identity(data_pd_y):
    """
    [description]
    : encode identity labels in a pandas dataframe
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    [return]
    : data_identity_onehot_y: numpy array, onehot encoding for identity labels
    """
    #
    ##
    data_location_pd_y = data_pd_y[["user_1_location", "user_2_location", 
                                    "user_3_location", "user_4_location", 
                                    "user_5_location", "user_6_location"]]
    # 
    data_identity_y = data_location_pd_y.to_numpy(copy = True).astype(str)
    #
    data_identity_y[data_identity_y != "nan"] = 1
    data_identity_y[data_identity_y == "nan"] = 0
    #
    data_identity_onehot_y = data_identity_y.astype("int8")
    #
    return data_identity_onehot_y

#
##
def encode_activity(data_pd_y, 
                    var_encoding):
    """
    [description]
    : encode activity labels in a pandas dataframe
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    : var_encoding: dict, encoding of different activities
    [return]
    : data_activity_onehot_y: numpy array, onehot encoding for activity labels
    """
    #
    ##
    data_activity_pd_y = data_pd_y[["user_1_activity", "user_2_activity", 
                                    "user_3_activity", "user_4_activity", 
                                    "user_5_activity", "user_6_activity"]]
    #
    data_activity_y = data_activity_pd_y.to_numpy(copy = True).astype(str)
    #
    data_activity_onehot_y = np.array([[var_encoding[var_y] for var_y in var_sample] for var_sample in data_activity_y])
    #
    return data_activity_onehot_y

#
##
def encode_location(data_pd_y, 
                    var_encoding):
    """
    [description]
    : encode location labels in a pandas dataframe
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    : var_encoding: dict, encoding of different locations
    [return]
    : data_location_onehot_y: numpy array, onehot encoding for location labels
    """
    #
    ##
    data_location_pd_y = data_pd_y[["user_1_location", "user_2_location", 
                                    "user_3_location", "user_4_location", 
                                    "user_5_location", "user_6_location"]]
    #
    data_location_y = data_location_pd_y.to_numpy(copy = True).astype(str)
    #
    data_location_onehot_y = np.array([[var_encoding[var_y] for var_y in var_sample] for var_sample in data_location_y])
    #
    return data_location_onehot_y

#
##
def test_load_data_y():
    """
    [description]
    : test load_data_y() function
    """
    #
    ##


#
##
def test_load_data_x():
    """
    [description]
    : test load_data_x() function
    """
    #
    ##
    data_pd_y = load_data_y(preset["path"]["data_y"],
                            var_environment = ["meeting_room"], 
                            var_wifi_band = ["2.4"], 
                            var_num_users = None)
    #
    var_label_list = data_pd_y["label"].to_list()
    #
    data_x = load_data_x(preset["path"]["data_x"], var_label_list)
    #
  

#
##
def test_encode_identity():
    """
    [description]
    : test encode_identity() function
    """
    #
    ##
    data_pd_y = pd.read_csv(preset["path"]["data_y"], dtype = str)
    #
    data_identity_onehot_y = encode_identity(data_pd_y)
    #


#
##
def test_encode_activity():
    """
    [description]
    : test encode_activity() function
    """
    #
    ##
    data_pd_y = pd.read_csv(preset["path"]["data_y"], dtype = str)
    #
    data_activity_onehot_y = encode_activity(data_pd_y, preset["encoding"]["activity"])
    #

#
##
def test_encode_location():
    """
    [description]
    : test encode_location() function
    """
    #
    ##
    data_pd_y = pd.read_csv(preset["path"]["data_y"], dtype = str)
    #
    data_location_onehot_y = encode_location(data_pd_y, preset["encoding"]["location"])
    #
   



train

In [4]:
import time
import torch
from torch import device
from torch.nn import Module
from torch.optim import Optimizer
from torch.utils.data import TensorDataset, DataLoader
from copy import deepcopy
from sklearn.metrics import accuracy_score


def train(
    model: Module,
    optimizer: Optimizer,
    loss: Module,
    data_train_set: TensorDataset,
    data_test_set: TensorDataset,
    var_threshold: float,
    var_batch_size: int,
    var_epochs: int,
    device: device,
    best_model_path: str = "best_model_lstm_epoch_300_lr_10-3_schaler.pth",
):
    """
    [description]
    : تابع آموزش (train) برای مدل‌های WiFi-based (مثلاً LSTM)
    
    [parameter]
    : model (Module): شیء مدل PyTorch
    : optimizer (Optimizer): بهینه‌ساز (مثلاً Adam)
    : loss (Module): تابع لاش (مثلاً BCEWithLogitsLoss)
    : data_train_set (TensorDataset): دیتاست آموزشی
    : data_test_set (TensorDataset): دیتاست آزمایشی
    : var_threshold (float): آستانه‌ی باینری کردن خروجی سیگموید
    : var_batch_size (int): سایز بچ
    : var_epochs (int): تعداد اپوک
    : device (device):cuda یا cpu
    : best_model_path (str): مسیر ذخیره‌ی بهترین مدل
    """

    data_train_loader = DataLoader(data_train_set, var_batch_size, shuffle=True, pin_memory=True)
    data_test_loader = DataLoader(data_test_set, batch_size=len(data_test_set), shuffle=False)

    var_best_accuracy = 0.0
    var_best_weight = None
    var_best_optimizer_state = None
    var_no_improvement_count = 0

    for var_epoch in range(var_epochs):
        var_time_e0 = time.time()

        # -------------------------- Train -----------------------------
        model.train()
        for data_batch in data_train_loader:
            data_batch_x, data_batch_y = data_batch
            data_batch_x = data_batch_x.to(device)
            data_batch_y = data_batch_y.to(device)

            predict_train_y = model(data_batch_x)
            var_loss_train = loss(
                predict_train_y, data_batch_y.reshape(data_batch_y.shape[0], -1).float()
            )

            optimizer.zero_grad()
            var_loss_train.backward()
            optimizer.step()

        # محاسبه‌ی دقت روی داده‌های آموزشی (برای نمایش)
        predict_train_y = (torch.sigmoid(predict_train_y) > var_threshold).float()
        data_batch_y = data_batch_y.detach().cpu().numpy()
        predict_train_y = predict_train_y.detach().cpu().numpy()

        predict_train_y = predict_train_y.reshape(-1, data_batch_y.shape[-1])
        data_batch_y = data_batch_y.reshape(-1, data_batch_y.shape[-1])
        var_accuracy_train = accuracy_score(data_batch_y.astype(int), predict_train_y.astype(int))

        # -------------------------- Evaluate ---------------------------
        model.eval()
        with torch.no_grad():
            data_test_x, data_test_y = next(iter(data_test_loader))
            data_test_x = data_test_x.to(device)
            data_test_y = data_test_y.to(device)

            predict_test_y = model(data_test_x)
            var_loss_test = loss(
                predict_test_y, data_test_y.reshape(data_test_y.shape[0], -1).float()
            )

            predict_test_y = (torch.sigmoid(predict_test_y) > var_threshold).float()

            data_test_y = data_test_y.detach().cpu().numpy()
            predict_test_y = predict_test_y.detach().cpu().numpy()

            predict_test_y = predict_test_y.reshape(-1, data_test_y.shape[-1])
            data_test_y = data_test_y.reshape(-1, data_test_y.shape[-1])
            var_accuracy_test = accuracy_score(data_test_y.astype(int), predict_test_y.astype(int))

        # ------------------------- Print log ---------------------------
        print(
            f"Epoch {var_epoch}/{var_epochs}",
            f"- {time.time() - var_time_e0:.6f}s",
            f"- Loss {var_loss_train.item():.6f}",
            f"- Accuracy {var_accuracy_train:.6f}",
            f"- Test Loss {var_loss_test.item():.6f}",
            f"- Test Accuracy {var_accuracy_test:.6f}",
        )

        # ------------------ Save best model & Early reduce lr ----------
        if var_accuracy_test > var_best_accuracy:
            var_best_accuracy = var_accuracy_test
            var_best_weight = deepcopy(model.state_dict())
            var_best_optimizer_state = deepcopy(optimizer.state_dict())
            torch.save({"best_weight": var_best_weight}, best_model_path)
            var_no_improvement_count = 0
        else:
            var_no_improvement_count += 1
            if var_no_improvement_count >= 30:
                for param_group in optimizer.param_groups:
                    param_group["lr"] /= 2
                var_no_improvement_count = 0

    # --------------- Save final (best) model & optimizer --------------
    # در اینجا صرفاً بهترین وضعیت مدل و اپتیمایزر را در فایل نهایی ذخیره می‌کنیم
    torch.save(
        {
            "best_weight": var_best_weight,
            "best_optimizer_state": var_best_optimizer_state,
        },
        best_model_path,
    )

    return var_best_weight



#model lstm

In [5]:
!pip install ptflops

In [6]:
"""
[file]          lstm.py
[description]   implement and evaluate WiFi-based model LSTM
"""
#
##
import time
import torch
import numpy as np
#
from torch.utils.data import TensorDataset
from ptflops import get_model_complexity_info
from sklearn.metrics import classification_report, accuracy_score
#



#
##
## ------------------------------------------------------------------------------------------ ##
## --------------------------------------- LSTM --------------------------------------------- ##
## ------------------------------------------------------------------------------------------ ##
class LSTMM(torch.nn.Module):
    #
    ##
    def __init__(self,
                 var_x_shape,
                 var_y_shape):
        #
        ##
        super(LSTMM, self).__init__()
        #
        var_dim_input = var_x_shape[-1]
        var_dim_output = var_y_shape[-1]
        #
        self.layer_norm = torch.nn.BatchNorm1d(var_dim_input)
        #
        self.layer_pooling = torch.nn.AvgPool1d(10, 10)
        #
        self.layer_lstm = torch.nn.LSTM(input_size = var_dim_input,
                                        hidden_size = 512, 
                                        batch_first = True)
        #
        self.layer_linear = torch.nn.Linear(512, var_dim_output)

    #
    ##
    def forward(self,
                var_input):
        #
        ##
        var_t = var_input
        #
        var_t = torch.permute(var_t, (0, 2, 1))
        var_t = self.layer_norm(var_t)
        var_t = self.layer_pooling(var_t)
        var_t = torch.permute(var_t, (0, 2, 1))
        #
        var_t, _ = self.layer_lstm(var_t)
        #
        var_t = var_t[:, -1, :]
        #
        var_t = self.layer_linear(var_t)
        #
        var_output = var_t
        #
        return var_output
    
#
##
def run_lstm(data_train_x, 
             data_train_y,
             data_test_x,
             data_test_y,
             var_repeat = 10):
    """
    [description]
    : run WiFi-based model LSTM
    [parameter]
    : data_train_x: numpy array, CSI amplitude to train model
    : data_train_y: numpy array, labels to train model
    : data_test_x: numpy array, CSI amplitude to test model
    : data_test_y: numpy array, labels to test model
    : var_repeat: int, number of repeated experiments
    [return]
    : result: dict, results of experiments
    """
    #
    ##
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #
    ##
    ## ============================================ Preprocess ============================================
    #
    ##
    data_train_x = data_train_x.reshape(data_train_x.shape[0], data_train_x.shape[1], -1)
    data_test_x = data_test_x.reshape(data_test_x.shape[0], data_test_x.shape[1], -1)
    #
    ## shape for model
    var_x_shape, var_y_shape = data_train_x[0].shape, data_train_y[0].reshape(-1).shape
    #
    data_train_set = TensorDataset(torch.from_numpy(data_train_x), torch.from_numpy(data_train_y))
    data_test_set = TensorDataset(torch.from_numpy(data_test_x), torch.from_numpy(data_test_y))
    #
    ##
    ## ========================================= Train & Evaluate =========================================
    #
    ##
    result = {}
    result_accuracy = []
    result_time_train = []
    result_time_test = []
    #
    ##
    var_macs, var_params = get_model_complexity_info(LSTMM(var_x_shape, var_y_shape), 
                                                     var_x_shape, as_strings = False)
    #
    print("Parameters:", var_params, "- FLOPs:", var_macs * 2)
    #
    ##
    for var_r in range(var_repeat):
        #
        ##
        print("Repeat", var_r)
        #
        torch.random.manual_seed(var_r + 39)
        #
        model_lstm = torch.compile(LSTMM(var_x_shape, var_y_shape).to(device))
        #
        optimizer = torch.optim.Adam(model_lstm.parameters(), 
                                     lr = preset["nn"]["lr"],
                                     weight_decay = 0)
        #
        loss = torch.nn.BCEWithLogitsLoss(pos_weight = torch.tensor([6] * var_y_shape[-1]).to(device))
        #
        var_time_0 = time.time()
        #
        ## ---------------------------------------- Train -----------------------------------------
        #
        var_best_weight = train(model = model_lstm, 
                                optimizer = optimizer, 
                                loss = loss, 
                                data_train_set = data_train_set,
                                data_test_set = data_test_set,
                                var_threshold = preset["nn"]["threshold"],
                                var_batch_size = preset["nn"]["batch_size"],
                                var_epochs = preset["nn"]["epoch"],
                                device = device)
        #
        var_time_1 = time.time()
        #
        ## ---------------------------------------- Test ------------------------------------------
        #
        model_lstm.load_state_dict(var_best_weight)
        #
        with torch.no_grad():
            predict_test_y = model_lstm(torch.from_numpy(data_test_x).to(device))
        #
        predict_test_y = (torch.sigmoid(predict_test_y) > preset["nn"]["threshold"]).float()
        predict_test_y = predict_test_y.detach().cpu().numpy()
        #
        var_time_2 = time.time()
        #
        ## -------------------------------------- Evaluate ----------------------------------------
        #
        ##
        data_test_y_c = data_test_y.reshape(-1, data_test_y.shape[-1])
        predict_test_y_c = predict_test_y.reshape(-1, data_test_y.shape[-1])
        #
        ## Accuracy
        result_acc = accuracy_score(data_test_y_c.astype(int), 
                                    predict_test_y_c.astype(int))
        #
        ## Report
        result_dict = classification_report(data_test_y_c, 
                                            predict_test_y_c, 
                                            digits = 6, 
                                            zero_division = 0, 
                                            output_dict = True)
        #
        result["repeat_" + str(var_r)] = result_dict
        #
        result_accuracy.append(result_acc)
        result_time_train.append(var_time_1 - var_time_0)
        result_time_test.append(var_time_2 - var_time_1)
        #
        print("repeat_" + str(var_r), result_accuracy)
        print(result)
    #
    ##
    result["accuracy"] = {"avg": np.mean(result_accuracy), "std": np.std(result_accuracy)}
    result["time_train"] = {"avg": np.mean(result_time_train), "std": np.std(result_time_train)}
    result["time_test"] = {"avg": np.mean(result_time_test), "std": np.std(result_time_test)}
    result["complexity"] = {"parameter": var_params, "flops": var_macs * 2}
    #
    return result

run

In [7]:
"""
[file]          run.py
[description]   run WiFi-based models
"""
#
##
import json
import argparse
from sklearn.model_selection import train_test_split
import torch._dynamo
torch._dynamo.config.suppress_errors = True

#




#


#
## parse arguments from input

#
var_task = "location"
var_model = "LSTM"
var_repeat = 2
#
## load annotation file as labels
data_pd_y = load_data_y(preset["path"]["data_y"],
                        var_environment = preset["data"]["environment"],
                        var_wifi_band = preset["data"]["wifi_band"],
                        var_num_users = preset["data"]["num_users"])
#
var_label_list = data_pd_y["label"].to_list()
#
## load CSI amplitude
data_x = load_data_x(preset["path"]["data_x"], var_label_list)
#
## encode labels
data_y = encode_data_y(data_pd_y, var_task)


## a training set (80%) and a test set (20%)
data_train_x, data_test_x, data_train_y, data_test_y = train_test_split(data_x, data_y,
                                                                        test_size = 0.2,
                                                                        shuffle = True,
                                                                        random_state = 39)
#
## select a WiFi-based model
# if var_model == "ST-RF": run_model = run_strf
# #
# if var_model == "MLP": run_model = run_mlp
#
if var_model == "LSTM": run_model = run_lstm
# #
# if var_model == "CNN-1D": run_model = run_cnn_1d
# #
# if var_model == "CNN-2D": run_model = run_cnn_2d
# #
# if var_model == "CLSTM": run_model = run_cnn_lstm
# #
# if var_model == "ABLSTM": run_model = run_ablstm
# #
if var_model == "THAT": run_model = run_that
#
## run WiFi-based model
result = run_model(data_train_x, data_train_y,
                   data_test_x, data_test_y, var_repeat)
#
##
result["model"] = var_model
result["task"] = var_task
result["data"] = preset["data"]
result["nn"] = preset["nn"]
#
print(result)
#
## save results
var_file = open(preset["path"]["save"], 'w')
json.dump(result, var_file, indent = 4)



LSTMM(
  1.62 M, 100.000% Params, 485.67 MMac, 99.833% MACs, 
  (layer_norm): BatchNorm1d(540, 0.033% Params, 1.62 MMac, 0.333% MACs, 270, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_pooling): AvgPool1d(0, 0.000% Params, 810.0 KMac, 0.167% MACs, kernel_size=(10,), stride=(10,), padding=(0,))
  (layer_lstm): LSTM(1.61 M, 99.018% Params, 483.23 MMac, 99.331% MACs, 270, 512, batch_first=True)
  (layer_linear): Linear(15.39 k, 0.949% Params, 15.39 KMac, 0.003% MACs, in_features=512, out_features=30, bias=True)
)
Parameters: 1621562 - FLOPs: 972961980
Repeat 0


W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125] WON'T CONVERT forward <ipython-input-6-600dd44513d4> line 48 
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125] due to: 
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125] Traceback (most recent call last):
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/output_graph.py", line 1446, in _call_user_compiler
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125]     compiled_fn = compiler_fn(gm, self.example_inputs())
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/_dynamo/repro/after_dynamo.py", line 129, in __call__
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125]     compiled_gm = compiler_fn(gm, example_inputs)
W0117 05:47:46.238000 18 torch/_dynamo/convert_frame.py:1125]   File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", l

Epoch 0/300 - 8.910580s - Loss 0.877786 - Accuracy 0.453125 - Test Loss 0.859980 - Test Accuracy 0.480548
Epoch 1/300 - 4.651071s - Loss 0.805523 - Accuracy 0.460069 - Test Loss 0.783675 - Test Accuracy 0.488506
Epoch 2/300 - 4.683573s - Loss 0.739113 - Accuracy 0.494792 - Test Loss 0.750276 - Test Accuracy 0.471706
Epoch 3/300 - 4.611439s - Loss 0.630379 - Accuracy 0.536458 - Test Loss 0.731456 - Test Accuracy 0.488948
Epoch 4/300 - 4.650517s - Loss 0.618249 - Accuracy 0.552083 - Test Loss 0.716777 - Test Accuracy 0.502653
Epoch 5/300 - 4.968754s - Loss 0.692940 - Accuracy 0.482639 - Test Loss 0.721527 - Test Accuracy 0.472591
Epoch 6/300 - 4.699270s - Loss 0.642846 - Accuracy 0.524306 - Test Loss 0.710197 - Test Accuracy 0.480106
Epoch 7/300 - 4.585065s - Loss 0.562256 - Accuracy 0.546875 - Test Loss 0.697629 - Test Accuracy 0.466401
Epoch 8/300 - 4.580285s - Loss 0.557331 - Accuracy 0.546875 - Test Loss 0.711896 - Test Accuracy 0.458886
Epoch 9/300 - 4.709992s - Loss 0.659191 - Accu